## Konkordansanalyse og topic modelling

Litt boilerplate import

In [20]:
import topicmodelling as tm
import imp
import util
import sqlite3
import numpy as np
from collections import Counter
from IPython.display import HTML, display
import pandas as pd

#### Finn filer

Skriv inn filnavn i listen under. Ved aktivering bygges en frekvensliste **freqs** og et format på filene ligger i variabelen **korpus**

In [6]:
korpus = tm.files2corpus(
    [
        "c:/users/larsj/downloads/frode_jøde_1918_1940.html", 
        "c:/users/larsj/Downloads/frode_jøde_extra_1918_19402.html"
    ]
)

print('korpus til filer ok')

freqs = tm.corpusfreq(korpus)



korpus til filer ok


### Sjekk de meste frekvente 

In [ ]:
freqs.most_common(20)

In [2]:
def query(db, sql, param = (), size=10):
    import sqlite3
    import sys
    res = []
    if type(sql) is str:
        sqls = sql.split(';')
    else:
        print('sql?')
        return
    with sqlite3.connect(db) as con:
        cur = con.cursor()
        try:
            for sql in sqls[:-1]:
                cur.execute(sql)
            cur.execute(sqls[-1], param)
            if size <= 0:
                res = cur.fetchall()
            else:
                res = cur.fetchmany(size)
        except:
            print('en feil oppstod:', str(sys.exc_info()[1]) )
    return res

### Frekvenslister (unigram) for bøker og aviser bør ligge et sted

In [3]:
avis_ngram = "c:/users/larsj/Downloads/avis-unigram-one-row.db"

In [7]:
diff = []
for word in freqs:
    if freqs[word] > 1:
        glob = query(avis_ngram, "select first, freq from unigram where first = ?",(word,))
        if glob != []:
            diff.append(glob[0])
dict(diff)

{'bort': 2530501,
 'brune': 152749,
 'flytte-biler': 19,
 'hugget': 46714,
 'iherdig': 69611,
 'undersøkte': 52893,
 'kirkens': 157586,
 'armé': 35679,
 'ilden': 179056,
 'nasjonalflagg': 1268,
 'Bryggearbeidernes': 335,
 'hoff': 24176,
 'papir': 290122,
 'fulgte': 690292,
 'opslkt': 1149,
 'krediteres': 4745,
 'Barnet': 66473,
 'vinterhjelpen': 218,
 'siden-': 1657,
 'lugtet': 579,
 'Rathenau': 779,
 'Savabini': 626,
 'togene': 46624,
 'påfølgende': 121978,
 'trusel': 72438,
 'dyret': 104339,
 'assimilere': 1052,
 '1935': 226396,
 'Polske': 32673,
 'ølne': 8241,
 'behandledes': 16683,
 'tilom': 237,
 'Le': 603359,
 'øvrige': 992197,
 'hodet': 764791,
 'brokede': 1042,
 'ngen': 177683,
 'uomvendte': 460,
 'jødeforfølgelser': 1439,
 'kjører': 643551,
 'Myrstad': 28382,
 'slor': 73446,
 'trekke': 834723,
 'trak': 33547,
 'Vossische': 1730,
 'guder': 26054,
 'André': 436403,
 'påtatt': 61254,
 'Drama': 331608,
 'aktivitet': 602314,
 'Olene': 7678,
 'Hel': 183865,
 'RØSBERG': 52,
 'Canada'

In [11]:
tot = Counter(dict(diff))

### Inspiser verdiene

In [18]:
HTML("""<table>
    <tr>
        <th>Total</th>
        <th>Freqs</th>
    </tr>
    <tr>
        <th>{tot}</th>
        <th>{freqs}</th>
        
    </tr>
</table>""".format(tot = sum(tot.values()), freqs= sum(freqs.values())))


Total,Freqs
17027473771,427167


### Fra tallene lages datarammer

In [21]:
corpus = pd.DataFrame.from_dict(freqs, orient='index')
totals = pd.DataFrame.from_dict(tot, orient='index')

In [22]:
corpus_rel = corpus.apply(lambda x: x/sum(freqs.values()))
totals_rel = totals.apply(lambda x: x/sum(tot.values()))

In [23]:
new = corpus_rel.merge(totals_rel,how='outer', left_index=True, right_index=True, sort=False)

In [70]:
pdnew = pd.DataFrame(np.log2(new['0_x']/new['0_y']))

In [56]:
pdnew.sort_values(by=0, ascending=False)[:20]

,0
nasjonaltyske,245.517562
riksborger,244.512451
statsborgerskåp,242.510864
Heksetog,239.887972
Storbiritannia,235.162857
Riabinin,235.162857
formuestelling,232.154430
Ogog,229.463300
Fasael,229.463300
schachtet,227.392973


In [28]:
def check_words(df, words):
    return df.loc[words].sort_values(by=0, ascending=False)

In [71]:
check_words(pdnew, 'fascister bolsjeviker kommunister nazister demokrater regjeringer'.split())

,0
bolsjeviker,7.167431
nazister,5.831036
kommunister,5.408229
fascister,5.049085
demokrater,4.037509
regjeringer,1.044061


In [73]:
check_words(pdnew, "Norge Sverige Storbritannia Tyskland Russland Frankrike Danmark Polen Finland Amerika".split())

,0
Tyskland,4.580114
Polen,3.655827
Amerika,2.431462
Russland,1.743338
Frankrike,0.955590
Storbritannia,0.201019
Danmark,-0.257921
Sverige,-0.808658
Finland,-1.099248
Norge,-1.353216


## Bruker eksisterende stoppordliste

In [5]:
stop_words = tm.stop_words

In [7]:
stop_words = dict({x:freqs[x] for (x,freqs[x]) in freqs.most_common(100) })

## Topic model NMF (~ Mallet)

**Modeller tema/topic**

Modifisert kode fra fra http://derekgreene.com/nmf-topic/

Aktiver cellen under - den definerer et skript for å utføre modelleringen

Ved temamodellering mallet-style må man bestemme seg for antallet tema, i tillegg til at teksten må brytes opp i mindre biter av en gitt størrelse. En liste over stoppord må også angis. Parametrene spesifiseres i kallet til tm.modeller() eller tm.modeller_set() på den her måten:

<p style="font-family:courier">tm.modeller(**teksten**, *tema*=**10**, *chunk_size*=**1000**, *topic_size*=**20**, *stop_words*=**stop_words**)</p>
<p style="font-family:courier">tm.modeller_set(**liste_av_tekster**, *tema*=**10**, *chunk_size*=**1000**, *topic_size*=**20**, *stop_words*=**stop_words**)</p>

Forskjellen på de to er at med *modeller()* skal teksten, det som er første argument, være en eneste tekst, mens for *modeller_set()* er det flere tekster. For eksempel, er *my_corpus* definert som i de første cellene, kan 

<p style="font-family:courier">tm.modeller(my_corpus[3], *tema*=**10**, *chunk_size*=**1000**, *topic_size*=**20**, *stop_words*=**stop_words**)</p>

være et kall, mens for *modeller_set* brukes referansen til hele listen:
<p style="font-family:courier">tm.modeller_set(my_corpus, *tema*=**10**, *chunk_size*=**1000**, *topic_size*=**20**, *stop_words*=**stop_words**)</p>

Se de to neste eksemplene

**Her modelleres selve Fengselslovutvalget**

In [12]:
tm.modeller(mitt_korpus[0], tema=20, chunk_size=50, stop_words = stop_words, topic_size=15)

['Berlin, blitt, få, får, siste, inn, arrestert, landet, nye, Fra, hvis, Man, Alle, Wien, lov',
 'Øog, jo, „, aa, saa, vært, siden, Jesus, Jerusalem, både, Alle, oss, Hitler, dog, fikk',
 'paa, saa, ogsaa, russiske, nogen, aa, gamle, os, sit, af, faa, end, aar, fik, gaar',
 '2, 3, ^, 4, kristen, 5, /, E, Gojim, ¦, 3., l, side, nye, Lord',
 'kristne, muhamedanere, kristen, På, Vi, op, igjen, finnes, både, kirke, 3, heller, går, mens, hittil',
 'drept, såret, to, Jerusalem, muhamedanere, blitt, sammenstøt, 20, 8, alvorlig, 45, britiske, 10, flere, 4',
 'fordi, hans, tilbake, folk, s, noen, tysk, unge, navn, alt, vært, blitt, både, hvad, gjøre',
 'ham, evige, kom, Jeg, to, op, hans, sa, Da, navn, r, fikk, hun, lille, Hovedr',
 'blandt, særlig, hjem, både, side, araberne, sitt, ta, arbeide, mest, arbeider, norske, annen, mere, arbeidet',
 'kjære, gammel, stille, aar, mor, Min, f, far, /, vår, søster, Oslo, vor, Stavanger, t',
 'hele, verden, folk, hjem, vel, Der, hver, står, gamle, store, 

** Her er noen konkordanser også **

In [39]:
tm.conc(mitt_korpus[0], 'Kvinde', num_of_result=50, offset=0)

['disse Raa er den Kvinde , som i foreliggende Sog',
 'helt igjennem ben æble Kvinde oærbig og i et Par',
 'Thi hun var en Kvinde , fom ved sin ualmindelige',
 'Keiserinden og Keiserprindsen fom Kvinde og Barn , men kun',
 'at enhver Mand og Kvinde herefter skal » ære fuldmyndig',
 'nemlig , at ugift Kvinde over 21 Aar , Enker',
 ', at myndig ugift Kvinde kan faa Sagnærge . Man',
 '. Naar da en Kvinde en Tid i Sønbom er',
 'Undersogelser , og en Kvinde , fra hvem Rygtet skulde',
 'ogfaa Sogllmneldelse- Den unge Kvinde af James . Ulv .',
 'vedkommende " Mand og Kvinde for Notarius puTjliciis ( 9',
 ', af Mand og Kvinde ihukommes med Hjertens © læbe',
 'lige for Mand og Kvinde , føltes dette stemt for',
 'at finde en velskikket Kvinde til Bestyrerinde ; thi det',
 'er ikke Mand efter Kvinde " , © at .',
 'en Mand og en Kvinde , som scmledes solges ab',
 'flere Kvinder eller en Kvinde og flere Mcend fom følge',
 ') til ben samaritanske Kvinde hist ved Jacobs Vrond kan',
 'Verden fodt af e

In [58]:
?tm.conc
